In [1]:
from bitcoinutils.keys import P2shAddress, PrivateKey, PublicKey
from bitcoinutils.script import Script
from bitcoinutils.setup import setup
from bitcoinutils.hdwallet import HDWallet
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput

setup("testnet")


for i in range(2):
    priv = PrivateKey()
    print("\nPrivate key WIF:", priv.to_wif(compressed=True))
    pub = priv.get_public_key()
    print("Public key:", pub.to_hex(compressed=True))


Private key WIF: cRApDi2t1tkdJkRcibmgvejGKujbZAETRKBQfPmn1TyRaytkjiVH
Public key: 02280eb108c1dd6780ad1ffaf3b2e8f9610edc29de96a5187c763139cace2bbc31

Private key WIF: cMynEiAp9LiutqCeMJR5phSp1zEgaM5cnGKG1a2mhwXB94TnX5wQ
Public key: 0302a04faa9274f2eec6a05fc4994a3484a7340c0feb1828df30b824d16f306671


In [2]:
#
# Send from a P2SH(P2WSH(P2PK)) UTXO to a P2PKH UTXO
#

# xprivkey = (
#     "tprv8ZgxMBicQKsPdQR9RuHpGGxSnNq8Jr3X4WnT6Nf2eq7FajuXyBep5KWYpYEixxx5XdTm1N"
#     "tpe84f3cVcF7mZZ7mPkntaFXLGJD2tS7YJkWU"
# )
# path = "m/86'/1'/0'/0/20"
# hdw = HDWallet(xprivkey, path)
# # from_priv = hdw.get_private_key()

# Private Key from generated in above code cell
from_priv = PrivateKey("cRApDi2t1tkdJkRcibmgvejGKujbZAETRKBQfPmn1TyRaytkjiVH")
print(from_priv.to_wif())
from_pub = from_priv.get_public_key()
from_addr = from_pub.get_address()

witness_script = Script([from_pub.to_hex(), "OP_CHECKSIG"])
p2sh_redeem_script = witness_script.to_p2wsh_script_pub_key()
print("From address:", P2shAddress.from_script(p2sh_redeem_script).to_string())

# hdw.from_path("m/86'/1'/0'/0/25")
# to_priv = hdw.get_private_key()
# Private Key2 from generated in above code cell
to_priv = PrivateKey("cMynEiAp9LiutqCeMJR5phSp1zEgaM5cnGKG1a2mhwXB94TnX5wQ")
to_address = to_priv.get_public_key().get_address()
print("To address:", to_address.to_string())

cRApDi2t1tkdJkRcibmgvejGKujbZAETRKBQfPmn1TyRaytkjiVH
From address: 2MuxtgnkyrUSudv99YbeApZQtfQmtuw7e8p
To address: n21JKC1dxNbfkjD1igojCuuxzFhUidj3sE


In [3]:
# UTXO's info
# Generated the transaction Id
txid = "7b7ca4e014cb699dffbc4b02e76c01850c7e2f6deefdb631dad1b9a16447fa08"
vout = 0
amount = 5000

txin = TxInput(txid, vout)
txout = TxOutput(3000, to_address.to_script_pub_key())
tx = Transaction([txin], [txout], has_segwit=True)

sig = from_priv.sign_segwit_input(tx, 0, witness_script, amount)

txin.script_sig = Script([p2sh_redeem_script.to_hex()])
tx.witnesses.append(TxWitnessInput([sig, witness_script.to_hex()]))

signed_tx = tx.serialize()

print(signed_tx)

0200000000010108fa4764a1b9d1da31b6fdee6d2f7e0c85016ce7024bbcff9d69cb14e0a47c7b00000000232200205a4a5b70ee8d94da712f8bf8eea004c30770a2b35a792eeab9300e68ab3e9caaffffffff01b80b0000000000001976a914e0bedfc69492883b4e997d965e86b95a7c905aae88ac02473044022009846fe3609748aeaf444c5c2fd3a6058af3596a6b48a485ea602ca408b4c3ba0220022b7ac3be8d12481be91e6144e7b0abf234549f1ca954b9b851362fac23989f01232102280eb108c1dd6780ad1ffaf3b2e8f9610edc29de96a5187c763139cace2bbc31ac00000000
